In [351]:
import ccxt
import pandas as pd
import json

with open('keys.json', 'r') as f:
    keys = json.load(f)
api_key = keys['api_key']
api_secret = keys['api_secret']

exchange = ccxt.binance({
    'apiKey': api_key,
    'secret': api_secret,
    'enableRateLimit': True
})

In [350]:
with open('keys.json', 'r') as f:
    keys = json.load(f)
api_key = keys['api_key']
api_secret = keys['api_secret']

In [352]:
exchange.set_sandbox_mode(True)

In [104]:
exchange.describe()

{'id': 'binance',
 'name': 'Binance',
 'countries': ['JP', 'MT'],
 'enableRateLimit': True,
 'rateLimit': 50,
 'timeout': 10000,
 'certified': True,
 'pro': True,
 'alias': False,
 'dex': False,
 'has': {'publicAPI': True,
  'privateAPI': True,
  'CORS': None,
  'sandbox': True,
  'spot': True,
  'margin': True,
  'swap': True,
  'future': True,
  'option': True,
  'addMargin': True,
  'borrowCrossMargin': True,
  'borrowIsolatedMargin': True,
  'borrowMargin': None,
  'cancelAllOrders': True,
  'cancelAllOrdersWs': None,
  'cancelOrder': True,
  'cancelOrderWs': None,
  'cancelOrders': True,
  'cancelOrdersWs': None,
  'closeAllPositions': False,
  'closePosition': False,
  'createDepositAddress': False,
  'createLimitBuyOrder': True,
  'createLimitBuyOrderWs': None,
  'createLimitOrder': True,
  'createLimitOrderWs': None,
  'createLimitSellOrder': True,
  'createLimitSellOrderWs': None,
  'createMarketBuyOrder': True,
  'createMarketBuyOrderWs': None,
  'createMarketBuyOrderWithCost

In [358]:
import ccxt
import pandas as pd

def retrieve_candles(symbol, timeframe, limit):
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)

    df_min = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df_min['timestamp'] = pd.to_datetime(df_min['timestamp'], unit='ms')
    return df_min

In [356]:
retrieve_candles('ETH/USDT', '1m', 100)

timestamp     open     high      low    close    volume
0  2024-08-31 07:27:00  2520.92  2520.92  2518.40  2519.40  119.7187
1  2024-08-31 07:28:00  2519.40  2520.22  2519.39  2519.69   50.5320
2  2024-08-31 07:29:00  2519.69  2519.69  2518.20  2519.05   74.8367
3  2024-08-31 07:30:00  2519.04  2520.01  2519.04  2519.50   27.6863
4  2024-08-31 07:31:00  2519.51  2519.93  2519.40  2519.92   79.7387
..                 ...      ...      ...      ...      ...       ...
95 2024-08-31 09:02:00  2516.29  2516.78  2515.98  2516.01   43.3399
96 2024-08-31 09:03:00  2516.01  2518.20  2516.01  2518.20   69.7580
97 2024-08-31 09:04:00  2518.26  2518.50  2518.00  2518.00   29.0797
98 2024-08-31 09:05:00  2518.00  2519.21  2517.60  2518.83   48.7600
99 2024-08-31 09:06:00  2518.83  2519.50  2518.48  2518.59   38.4994

[100 rows x 6 columns]

In [357]:
import plotly.graph_objects as go

import pandas as pd
from datetime import datetime

fig = go.Figure(data=[go.Candlestick(x=df_min['timestamp'],
                open=df_min['open'],
                high=df_min['high'],
                low=df_min['low'],
                close=df_min['close'])])
fig.update_layout(
    width=900,
    height=700)
fig.show()

In [341]:
if 'order_book' in globals():
    del order_book

column_names = ['Date', 'Time', 'Symbol', 'Side', 'Price', 'Quantity', 'Value', 'executedQty', 'Commission']

order_book = pd.DataFrame(columns=column_names)
order_book

Empty DataFrame
Columns: [Date, Time, Symbol, Side, Price, Quantity, Value, executedQty, Commission]
Index: []

In [342]:
def append_order_book(order):
    for item in order['info']['fills']:
        order_book.loc[len(order_book.index)] = [order['datetime'].split('T')[0],
                        order['datetime'].split('T')[1][:-5],
                        order['info']['symbol'],
                        order['info']['side'],
                        item['price'],
                        item['qty'],
                        float(item['price'])*float(item['qty']),
                        order['info']['executedQty'],
                        item['commission']]
    return order_book

In [343]:
def place_order(signal, amount, symbol):
    if signal == 'buy':
        order = exchange.create_market_buy_order(symbol, amount)
    elif signal == 'sell':
        order = exchange.create_market_sell_order(symbol, amount)
    order_book = append_order_book(order)
    order_book = pd.DataFrame(order_book)
    return order_book

In [345]:
place_order(signal = 'buy', amount = '0.4', symbol = 'ETH/USDT')

Date      Time   Symbol Side          Price    Quantity       Value  \
0  2024-08-31  08:04:25  ETHUSDT  BUY  2522.52000000  0.10910000  275.206932   
1  2024-08-31  08:04:25  ETHUSDT  BUY  2522.58000000  0.19830000  500.227614   
2  2024-08-31  08:04:25  ETHUSDT  BUY  2522.59000000  0.09260000  233.591834   

  executedQty  Commission  
0  0.40000000  0.00000000  
1  0.40000000  0.00000000  
2  0.40000000  0.00000000

In [339]:
if 'current_assets' in globals():
    del current_assets

column_names = ['Asset', 'Free Value', 'Locked Value', 'Value']

current_assets = pd.DataFrame(columns=column_names)
current_assets

Empty DataFrame
Columns: [Asset, Free Value, Locked Value, Value]
Index: []

In [319]:
import numpy as np

def get_current_price(symbol):
    try:
        ticker = exchange.fetch_ticker(symbol)
        
        current_price = ticker.get('last', np.nan)
        return current_price
    except:
        return np.nan

In [321]:
import numpy as np

for item in balance['info']['balances'][:10]:
    if item['asset'] == 'USDT':
        value = item['free']
    else:
        symbol = f"{item['asset']}/USDT"
        current_price = get_current_price(symbol)
        value = float(item['free'])*float(current_price)

    current_assets.loc[len(current_assets.index)] = [item['asset'], item['free'], item['locked'], value]
    
current_assets

Asset     Free Value Locked Value          Value
0     ETH     0.64120000   0.00000000     1614.41336
1     BTC     1.12300000   0.00000000     66354.3641
2     LTC     8.00000000   0.00000000         520.88
3     BNB     1.00000000   0.00000000          534.5
4    USDT  3630.00161660   0.00000000  3630.00161660
..    ...            ...          ...            ...
493   TRX  4026.00000000   0.00000000       642.5496
494   XRP   973.00000000   0.00000000       551.3991
495   NEO    55.00000000   0.00000000          568.7
496  QTUM   239.00000000   0.00000000        561.172
497   EOS  1095.00000000   0.00000000       535.3455

[498 rows x 4 columns]